In [20]:
import os
import gc
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
import utils
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dir = '../input'
!ls ../input

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
train_label = pd.read_csv(os.path.join(data_dir, 'train_v2.csv'))
labels_str = 'agriculture, artisinal_mine, bare_ground, blooming, blow_down, clear, cloudy, conventional_mine, cultivation, habitation, haze, partly_cloudy, primary, road, selective_logging, slash_burn, water'
labels = labels_str.split(', ')
label_map = {x: labels.index(x) for x in labels}

In [4]:
def tags_to_vec(tags):
    tags_list = tags.split(' ')
    vec = np.zeros(17)
    for t in tags_list:
        vec[label_map[t]] = 1
    return vec

In [5]:
train_label['y'] = train_label.tags.apply(tags_to_vec)

In [27]:
X_train = []
y_train = []
N_train_limit = 2000e9
i=0
for idx, row in tqdm(train_label.iterrows(), total=min(N_train_limit, train_label.shape[0])):
    image = cv2.imread(os.path.join('../input', 'train-jpg', '{}.jpg'.format(row['image_name'])))
    #image = image[:,:,::-1]
    #plt.imshow(image)
    X_train.append(image)
    y_train.append(row['y'])
    i+=1
    if i==N_train_limit:
        break

100%|███████████████████████████████████████████████████████████████████████████| 40479/40479 [01:06<00:00, 609.76it/s]


In [28]:
X_train = np.stack(X_train, axis=0)
y_train = np.stack(y_train, axis=0)
print(X_train.shape, y_train.shape)

(40479, 256, 256, 3) (40479, 17)


In [29]:
xtrain, xvalid, ytrain, yvalid = train_test_split(X_train, y_train, test_size = 0.2)
print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(32383, 256, 256, 3) (8096, 256, 256, 3) (32383, 17) (8096, 17)


In [30]:
try:
    del X_train, y_train
except:
    pass
gc.collect()

1574

In [31]:
xtrain = xtrain.astype('float32')
xvalid = xvalid.astype('float32')
xtrain /= 255
xvalid /= 255

In [32]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [33]:
batch_size = 32
num_classes = 17
epochs = 200
data_augmentation = True

In [34]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(17, activation='sigmoid', dro)(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='binary_crossentropy')

In [35]:
# defining a set of callbacks
class f2beta(Callback):
    def __init__(self, xval, yval):
        self.xval = xval
        self.yval = yval
        self.maps = []

    def eval_map(self):
        x_val = self.xval
        y_true = self.yval
        y_pred = self.model.predict(x_val)
        y_pred = (y_pred > 0.2).astype(int)
        score = fbeta_score(y_true, y_pred, beta=2, average='samples')
        return score

    def on_epoch_end(self, epoch, logs={}):
        score = self.eval_map()
        print("f2beta for epoch %d is %f" %(epoch, score))
        self.maps.append(score)

beta_score = f2beta(xvalid, yvalid)

checkpoint = ModelCheckpoint("weights.{epoch:02d}-{val_loss:.5f}.hdf5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

In [36]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(xtrain, ytrain,
              batch_size=batch_size,
              epochs=epochs,callbacks=[checkpoint, beta_score, earlystop],
              validation_data=(xvalid, yvalid),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(xtrain)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(xtrain, ytrain,
                                     batch_size=batch_size),
                        steps_per_epoch=xtrain.shape[0] // batch_size,
                        epochs=epochs,callbacks=[checkpoint, beta_score, earlystop],
                        validation_data=(xvalid, yvalid))

Using real-time data augmentation.
Epoch 1/200


 312/1011 [========>.....................] - ETA: 1942s - loss: 0.65 - ETA: 1168s - loss: 0.50 - ETA: 910s - loss: 0.4536 - ETA: 780s - loss: 0.419 - ETA: 702s - loss: 0.397 - ETA: 650s - loss: 0.383 - ETA: 612s - loss: 0.371 - ETA: 584s - loss: 0.357 - ETA: 562s - loss: 0.343 - ETA: 544s - loss: 0.342 - ETA: 530s - loss: 0.331 - ETA: 517s - loss: 0.321 - ETA: 507s - loss: 0.314 - ETA: 498s - loss: 0.307 - ETA: 490s - loss: 0.309 - ETA: 484s - loss: 0.304 - ETA: 477s - loss: 0.306 - ETA: 472s - loss: 0.302 - ETA: 467s - loss: 0.296 - ETA: 462s - loss: 0.291 - ETA: 458s - loss: 0.290 - ETA: 454s - loss: 0.287 - ETA: 451s - loss: 0.284 - ETA: 448s - loss: 0.283 - ETA: 445s - loss: 0.282 - ETA: 442s - loss: 0.280 - ETA: 439s - loss: 0.279 - ETA: 437s - loss: 0.277 - ETA: 435s - loss: 0.274 - ETA: 432s - loss: 0.272 - ETA: 430s - loss: 0.271 - ETA: 428s - loss: 0.268 - ETA: 427s - loss: 0.267 - ETA: 425s - loss: 0.267 - ETA: 423s - loss: 0.265 - ETA: 421s - loss: 0.263 - ETA: 420s - loss: 

 626/1011 [=================>............] - ETA: 277s - loss: 0.187 - ETA: 276s - loss: 0.187 - ETA: 276s - loss: 0.187 - ETA: 276s - loss: 0.187 - ETA: 275s - loss: 0.187 - ETA: 275s - loss: 0.187 - ETA: 275s - loss: 0.187 - ETA: 274s - loss: 0.187 - ETA: 274s - loss: 0.187 - ETA: 273s - loss: 0.186 - ETA: 273s - loss: 0.186 - ETA: 273s - loss: 0.186 - ETA: 272s - loss: 0.186 - ETA: 272s - loss: 0.186 - ETA: 271s - loss: 0.186 - ETA: 271s - loss: 0.186 - ETA: 270s - loss: 0.186 - ETA: 270s - loss: 0.186 - ETA: 270s - loss: 0.186 - ETA: 269s - loss: 0.186 - ETA: 269s - loss: 0.185 - ETA: 268s - loss: 0.185 - ETA: 268s - loss: 0.186 - ETA: 268s - loss: 0.185 - ETA: 267s - loss: 0.185 - ETA: 267s - loss: 0.185 - ETA: 266s - loss: 0.185 - ETA: 266s - loss: 0.185 - ETA: 266s - loss: 0.185 - ETA: 265s - loss: 0.185 - ETA: 265s - loss: 0.185 - ETA: 264s - loss: 0.185 - ETA: 264s - loss: 0.185 - ETA: 263s - loss: 0.185 - ETA: 263s - loss: 0.185 - ETA: 263s - loss: 0.185 - ETA: 262s - loss: 0

 955/1011 [===========================>..] - ETA: 151s - loss: 0.174 - ETA: 150s - loss: 0.174 - ETA: 150s - loss: 0.174 - ETA: 150s - loss: 0.174 - ETA: 149s - loss: 0.174 - ETA: 149s - loss: 0.174 - ETA: 148s - loss: 0.174 - ETA: 148s - loss: 0.174 - ETA: 148s - loss: 0.174 - ETA: 147s - loss: 0.174 - ETA: 147s - loss: 0.174 - ETA: 146s - loss: 0.174 - ETA: 146s - loss: 0.174 - ETA: 146s - loss: 0.174 - ETA: 145s - loss: 0.174 - ETA: 145s - loss: 0.174 - ETA: 144s - loss: 0.174 - ETA: 144s - loss: 0.174 - ETA: 144s - loss: 0.174 - ETA: 143s - loss: 0.174 - ETA: 143s - loss: 0.174 - ETA: 143s - loss: 0.174 - ETA: 142s - loss: 0.174 - ETA: 142s - loss: 0.174 - ETA: 141s - loss: 0.174 - ETA: 141s - loss: 0.174 - ETA: 141s - loss: 0.174 - ETA: 140s - loss: 0.174 - ETA: 140s - loss: 0.174 - ETA: 139s - loss: 0.173 - ETA: 139s - loss: 0.173 - ETA: 139s - loss: 0.173 - ETA: 138s - loss: 0.173 - ETA: 138s - loss: 0.173 - ETA: 137s - loss: 0.173 - ETA: 137s - loss: 0.173 - ETA: 137s - loss: 0

1010/1011 [============================>.] - ETA: 21s - loss: 0.16 - ETA: 21s - loss: 0.16 - ETA: 20s - loss: 0.16 - ETA: 20s - loss: 0.16 - ETA: 20s - loss: 0.16 - ETA: 19s - loss: 0.16 - ETA: 19s - loss: 0.16 - ETA: 18s - loss: 0.16 - ETA: 18s - loss: 0.16 - ETA: 18s - loss: 0.16 - ETA: 17s - loss: 0.16 - ETA: 17s - loss: 0.16 - ETA: 16s - loss: 0.16 - ETA: 16s - loss: 0.16 - ETA: 16s - loss: 0.16 - ETA: 15s - loss: 0.16 - ETA: 15s - loss: 0.16 - ETA: 14s - loss: 0.16 - ETA: 14s - loss: 0.16 - ETA: 14s - loss: 0.16 - ETA: 13s - loss: 0.16 - ETA: 13s - loss: 0.16 - ETA: 12s - loss: 0.16 - ETA: 12s - loss: 0.16 - ETA: 12s - loss: 0.16 - ETA: 11s - loss: 0.16 - ETA: 11s - loss: 0.16 - ETA: 11s - loss: 0.16 - ETA: 10s - loss: 0.16 - ETA: 10s - loss: 0.16 - ETA: 9s - loss: 0.1670 - ETA: 9s - loss: 0.167 - ETA: 9s - loss: 0.167 - ETA: 8s - loss: 0.167 - ETA: 8s - loss: 0.166 - ETA: 7s - loss: 0.166 - ETA: 7s - loss: 0.167 - ETA: 7s - loss: 0.166 - ETA: 6s - loss: 0.166 - ETA: 6s - loss: 0.

...

1010/1011 [============================>.] - ETA: 20s - loss: 0.14 - ETA: 20s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 18s - loss: 0.14 - ETA: 18s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 16s - loss: 0.14 - ETA: 16s - loss: 0.14 - ETA: 16s - loss: 0.14 - ETA: 15s - loss: 0.14 - ETA: 15s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 13s - loss: 0.14 - ETA: 13s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 11s - loss: 0.14 - ETA: 11s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 9s - loss: 0.1445 - ETA: 9s - loss: 0.144 - ETA: 8s - loss: 0.144 - ETA: 8s - loss: 0.144 - ETA: 8s - loss: 0.144 - ETA: 7s - loss: 0.144 - ETA: 7s - loss: 0.144 - ETA: 7s - loss: 0.144 - ETA: 6s - loss: 0.144 - ETA: 6s - loss: 0.144 - ETA: 5s - loss: 0.144 - ETA: 5s - loss: 0.

 314/1011 [========>.....................] - ETA: 400s - loss: 0.117 - ETA: 397s - loss: 0.138 - ETA: 390s - loss: 0.140 - ETA: 390s - loss: 0.141 - ETA: 390s - loss: 0.143 - ETA: 390s - loss: 0.144 - ETA: 390s - loss: 0.141 - ETA: 389s - loss: 0.140 - ETA: 389s - loss: 0.148 - ETA: 389s - loss: 0.147 - ETA: 389s - loss: 0.144 - ETA: 388s - loss: 0.140 - ETA: 388s - loss: 0.141 - ETA: 388s - loss: 0.143 - ETA: 387s - loss: 0.146 - ETA: 387s - loss: 0.144 - ETA: 386s - loss: 0.144 - ETA: 386s - loss: 0.144 - ETA: 385s - loss: 0.145 - ETA: 385s - loss: 0.144 - ETA: 385s - loss: 0.145 - ETA: 384s - loss: 0.144 - ETA: 384s - loss: 0.143 - ETA: 384s - loss: 0.143 - ETA: 383s - loss: 0.144 - ETA: 383s - loss: 0.146 - ETA: 382s - loss: 0.145 - ETA: 382s - loss: 0.145 - ETA: 382s - loss: 0.145 - ETA: 381s - loss: 0.144 - ETA: 381s - loss: 0.144 - ETA: 381s - loss: 0.143 - ETA: 380s - loss: 0.145 - ETA: 380s - loss: 0.144 - ETA: 379s - loss: 0.143 - ETA: 379s - loss: 0.144 - ETA: 379s - loss: 0

 628/1011 [=================>............] - ETA: 271s - loss: 0.141 - ETA: 270s - loss: 0.141 - ETA: 270s - loss: 0.141 - ETA: 269s - loss: 0.141 - ETA: 269s - loss: 0.141 - ETA: 269s - loss: 0.141 - ETA: 268s - loss: 0.141 - ETA: 268s - loss: 0.141 - ETA: 267s - loss: 0.141 - ETA: 267s - loss: 0.141 - ETA: 267s - loss: 0.141 - ETA: 266s - loss: 0.141 - ETA: 266s - loss: 0.141 - ETA: 266s - loss: 0.141 - ETA: 265s - loss: 0.141 - ETA: 265s - loss: 0.141 - ETA: 264s - loss: 0.141 - ETA: 264s - loss: 0.141 - ETA: 264s - loss: 0.141 - ETA: 263s - loss: 0.141 - ETA: 263s - loss: 0.141 - ETA: 262s - loss: 0.141 - ETA: 262s - loss: 0.141 - ETA: 262s - loss: 0.141 - ETA: 261s - loss: 0.140 - ETA: 261s - loss: 0.140 - ETA: 260s - loss: 0.141 - ETA: 260s - loss: 0.141 - ETA: 260s - loss: 0.141 - ETA: 259s - loss: 0.141 - ETA: 259s - loss: 0.141 - ETA: 258s - loss: 0.141 - ETA: 258s - loss: 0.141 - ETA: 258s - loss: 0.141 - ETA: 257s - loss: 0.141 - ETA: 257s - loss: 0.141 - ETA: 257s - loss: 0

 957/1011 [===========================>..] - ETA: 148s - loss: 0.140 - ETA: 148s - loss: 0.140 - ETA: 148s - loss: 0.140 - ETA: 147s - loss: 0.140 - ETA: 147s - loss: 0.140 - ETA: 146s - loss: 0.140 - ETA: 146s - loss: 0.140 - ETA: 146s - loss: 0.140 - ETA: 145s - loss: 0.140 - ETA: 145s - loss: 0.140 - ETA: 144s - loss: 0.140 - ETA: 144s - loss: 0.140 - ETA: 144s - loss: 0.140 - ETA: 143s - loss: 0.140 - ETA: 143s - loss: 0.140 - ETA: 142s - loss: 0.140 - ETA: 142s - loss: 0.140 - ETA: 142s - loss: 0.140 - ETA: 141s - loss: 0.140 - ETA: 141s - loss: 0.140 - ETA: 140s - loss: 0.140 - ETA: 140s - loss: 0.140 - ETA: 140s - loss: 0.140 - ETA: 139s - loss: 0.140 - ETA: 139s - loss: 0.140 - ETA: 139s - loss: 0.140 - ETA: 138s - loss: 0.140 - ETA: 138s - loss: 0.140 - ETA: 137s - loss: 0.140 - ETA: 137s - loss: 0.140 - ETA: 137s - loss: 0.140 - ETA: 136s - loss: 0.140 - ETA: 136s - loss: 0.140 - ETA: 135s - loss: 0.140 - ETA: 135s - loss: 0.140 - ETA: 135s - loss: 0.140 - ETA: 134s - loss: 0

1010/1011 [============================>.] - ETA: 20s - loss: 0.14 - ETA: 20s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 19s - loss: 0.14 - ETA: 18s - loss: 0.14 - ETA: 18s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 17s - loss: 0.14 - ETA: 16s - loss: 0.14 - ETA: 16s - loss: 0.14 - ETA: 15s - loss: 0.14 - ETA: 15s - loss: 0.14 - ETA: 15s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 14s - loss: 0.14 - ETA: 13s - loss: 0.14 - ETA: 13s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 12s - loss: 0.14 - ETA: 11s - loss: 0.14 - ETA: 11s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 10s - loss: 0.14 - ETA: 9s - loss: 0.1415 - ETA: 9s - loss: 0.141 - ETA: 8s - loss: 0.141 - ETA: 8s - loss: 0.141 - ETA: 8s - loss: 0.141 - ETA: 7s - loss: 0.141 - ETA: 7s - loss: 0.141 - ETA: 7s - loss: 0.141 - ETA: 6s - loss: 0.141 - ETA: 6s - loss: 0.141 - ETA: 5s - loss: 0.141 - ETA: 5s - loss: 0.

In [ ]:
# model.fit(xtrain, ytrain, epochs=10, batch_size=32, verbose=True)

In [ ]:
# train the model on the new data for a few epochs
model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)